# 🎯 RL-Based Prompt Optimization with Real LLM Calls

This notebook demonstrates **Reinforcement Learning** for dynamic prompt optimization
using **real LLM API calls** via **LangChain/LangGraph**.

## Architecture

```
┌──────────────────────────────────────────────────────────────────────────────┐
│              RL-BASED PROMPT OPTIMIZATION (Real LLM Calls)                   │
├──────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│  ┌─────────┐    ┌──────────────────┐    ┌──────────────────┐                │
│  │  User   │───►│  RL PROMPT AGENT  │───►│   LLM SERVICE    │                │
│  │  Query  │    │  (LangGraph)      │    │  (OpenAI /       │                │
│  └─────────┘    │                   │    │   Anthropic)     │                │
│       │         │ • Select Template │    │                  │                │
│       │         │ • Choose Tone     │    │ • Generate       │                │
│       │         │ • Set Parameters  │    │   Response       │                │
│       │         └────────┬──────────┘    └────────┬─────────┘                │
│       │                  │                        │                          │
│       │                  ▼                        ▼                          │
│       │         ┌──────────────────┐    ┌──────────────────┐                │
│       │         │  RL OPTIMIZER    │◄───│  LLM-AS-JUDGE    │                │
│       │         │                  │    │  REWARD MODEL     │                │
│       │         │ • Q-Learning     │    │                  │                │
│       │         │ • Thompson       │    │ • Quality Score  │                │
│       └────────►│   Sampling       │    │ • Relevance      │                │
│   Feedback      │ • UCB            │    │ • Helpfulness    │                │
│                 └──────────────────┘    │ • Cost Tracking  │                │
│                                         └──────────────────┘                │
└──────────────────────────────────────────────────────────────────────────────┘
```

## Key Differences from Simulated Version

| Aspect | Simulated | This Notebook |
|--------|-----------|---------------|
| **LLM Calls** | Simulated scores | Real OpenAI / Anthropic API calls |
| **Reward Signal** | Heuristic formula | LLM-as-Judge + token cost tracking |
| **Workflow** | Manual loops | LangGraph state machine |
| **Models** | None | GPT-4o-mini, Claude 3.5 Haiku |
| **Cost Tracking** | Estimated | Real token usage from API |

## Prerequisites

- `OPENAI_API_KEY` environment variable set
- `ANTHROPIC_API_KEY` environment variable set

## 1. Environment Setup

In [ ]:
# Install required packages
%pip install -q numpy pandas matplotlib seaborn scikit-learn \
    langchain langchain-openai langchain-anthropic langgraph \
    langchain-core pydantic tiktoken

In [ ]:
import os
import json
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Optional, Any, Literal, TypedDict
from dataclasses import dataclass, field
from enum import Enum
from collections import defaultdict
import random
from abc import ABC, abstractmethod
import warnings
warnings.filterwarnings('ignore')

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from pydantic import BaseModel, Field

# LangGraph imports
from langgraph.graph import StateGraph, END, START

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)

print("✅ All imports successful!")

In [ ]:
# Verify API keys are set
assert os.environ.get("OPENAI_API_KEY"), "❌ OPENAI_API_KEY not set!"
assert os.environ.get("ANTHROPIC_API_KEY"), "❌ ANTHROPIC_API_KEY not set!"
print("✅ API keys verified")
print(f"   OpenAI key: ...{os.environ['OPENAI_API_KEY'][-6:]}")
print(f"   Anthropic key: ...{os.environ['ANTHROPIC_API_KEY'][-6:]}")

## 2. LLM Service Layer

We set up LangChain models for both OpenAI and Anthropic, with token tracking for cost-aware optimization.

In [ ]:
class LLMProvider(Enum):
    """Available LLM providers."""
    OPENAI = "openai"
    ANTHROPIC = "anthropic"


@dataclass
class LLMResponse:
    """Structured response from an LLM call."""
    content: str
    provider: str
    model: str
    input_tokens: int
    output_tokens: int
    total_tokens: int
    latency_seconds: float
    cost_estimate: float  # Estimated cost in USD


class LLMService:
    """
    Manages LLM calls to OpenAI and Anthropic with token/cost tracking.
    """

    # Approximate pricing per 1M tokens (USD)
    PRICING = {
        "gpt-4o-mini": {"input": 0.15, "output": 0.60},
        "gpt-4o": {"input": 2.50, "output": 10.00},
        "claude-3-5-haiku-latest": {"input": 0.80, "output": 4.00},
        "claude-3-5-sonnet-latest": {"input": 3.00, "output": 15.00},
    }

    def __init__(
        self,
        openai_model: str = "gpt-4o-mini",
        anthropic_model: str = "claude-3-5-haiku-latest",
        temperature: float = 0.7,
        max_tokens: int = 1024,
    ):
        self.openai_model_name = openai_model
        self.anthropic_model_name = anthropic_model

        self.openai_llm = ChatOpenAI(
            model=openai_model,
            temperature=temperature,
            max_tokens=max_tokens,
        )
        self.anthropic_llm = ChatAnthropic(
            model=anthropic_model,
            temperature=temperature,
            max_tokens=max_tokens,
        )

        # Cumulative cost tracker
        self.total_cost = 0.0
        self.call_count = 0
        self.call_log: List[Dict[str, Any]] = []

    def _estimate_cost(self, model: str, input_tokens: int, output_tokens: int) -> float:
        """Estimate cost in USD for a single call."""
        pricing = self.PRICING.get(model, {"input": 1.0, "output": 3.0})
        cost = (input_tokens * pricing["input"] + output_tokens * pricing["output"]) / 1_000_000
        return cost

    def call(
        self,
        system_prompt: str,
        user_message: str,
        provider: LLMProvider = LLMProvider.OPENAI,
    ) -> LLMResponse:
        """
        Make a real LLM call and return structured response with token usage.
        """
        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=user_message),
        ]

        if provider == LLMProvider.OPENAI:
            llm = self.openai_llm
            model_name = self.openai_model_name
        else:
            llm = self.anthropic_llm
            model_name = self.anthropic_model_name

        start = time.time()
        result = llm.invoke(messages)
        latency = time.time() - start

        # Extract token usage from response metadata
        usage = result.usage_metadata or {}
        input_tokens = usage.get("input_tokens", 0)
        output_tokens = usage.get("output_tokens", 0)
        total_tokens = usage.get("total_tokens", input_tokens + output_tokens)

        cost = self._estimate_cost(model_name, input_tokens, output_tokens)
        self.total_cost += cost
        self.call_count += 1

        response = LLMResponse(
            content=result.content,
            provider=provider.value,
            model=model_name,
            input_tokens=input_tokens,
            output_tokens=output_tokens,
            total_tokens=total_tokens,
            latency_seconds=round(latency, 3),
            cost_estimate=cost,
        )

        self.call_log.append({
            "model": model_name,
            "input_tokens": input_tokens,
            "output_tokens": output_tokens,
            "latency": latency,
            "cost": cost,
        })

        return response

    def get_cost_summary(self) -> Dict[str, Any]:
        """Return cumulative cost summary."""
        if not self.call_log:
            return {"total_calls": 0, "total_cost": 0.0}
        df = pd.DataFrame(self.call_log)
        return {
            "total_calls": self.call_count,
            "total_cost_usd": round(self.total_cost, 6),
            "avg_latency_s": round(df["latency"].mean(), 3),
            "avg_input_tokens": int(df["input_tokens"].mean()),
            "avg_output_tokens": int(df["output_tokens"].mean()),
            "by_model": df.groupby("model")["cost"].sum().to_dict(),
        }


# Instantiate the service
llm_service = LLMService(
    openai_model="gpt-4o-mini",
    anthropic_model="claude-3-5-haiku-latest",
    temperature=0.7,
    max_tokens=512,
)

# Quick smoke test
test_resp = llm_service.call(
    system_prompt="You are a helpful assistant.",
    user_message="Say hello in one sentence.",
    provider=LLMProvider.OPENAI,
)
print("✅ LLM Service initialized and tested!")
print(f"   Response: {test_resp.content[:100]}")
print(f"   Tokens: {test_resp.total_tokens} | Latency: {test_resp.latency_seconds}s | Cost: ${test_resp.cost_estimate:.6f}")

## 3. Prompt Components and Templates

Same building blocks as the simulated version, but now each template produces a real system prompt sent to the LLM.

In [ ]:
class QueryType(Enum):
    """Types of user queries that require different prompt strategies."""
    FACTUAL = "factual"
    CREATIVE = "creative"
    ANALYTICAL = "analytical"
    CODING = "coding"
    CONVERSATIONAL = "conversational"


class ToneStyle(Enum):
    """Tone options for prompt responses."""
    FORMAL = "formal"
    CASUAL = "casual"
    TECHNICAL = "technical"
    FRIENDLY = "friendly"


class DetailLevel(Enum):
    """Level of detail in responses."""
    BRIEF = "brief"
    MODERATE = "moderate"
    DETAILED = "detailed"
    COMPREHENSIVE = "comprehensive"


@dataclass
class PromptTemplate:
    """A prompt template with configurable components."""
    id: str
    name: str
    system_instruction: str
    supports_examples: bool = True
    supports_chain_of_thought: bool = True

    def render_system_prompt(
        self,
        tone: ToneStyle = ToneStyle.FORMAL,
        detail: DetailLevel = DetailLevel.MODERATE,
        use_chain_of_thought: bool = False,
        examples: List[str] = None,
    ) -> str:
        """Build the full system prompt."""
        parts = [self.system_instruction]

        tone_map = {
            ToneStyle.FORMAL: "Use a professional and formal tone.",
            ToneStyle.CASUAL: "Use a casual and conversational tone.",
            ToneStyle.TECHNICAL: "Use precise technical language.",
            ToneStyle.FRIENDLY: "Be warm and approachable.",
        }
        parts.append(tone_map[tone])

        detail_map = {
            DetailLevel.BRIEF: "Keep your response concise (2-3 sentences).",
            DetailLevel.MODERATE: "Provide a balanced response with key details.",
            DetailLevel.DETAILED: "Provide a thorough and detailed response.",
            DetailLevel.COMPREHENSIVE: "Provide an exhaustive response covering all aspects.",
        }
        parts.append(detail_map[detail])

        if self.supports_examples and examples:
            parts.append("\nHere are some examples for reference:")
            for i, ex in enumerate(examples, 1):
                parts.append(f"  {i}. {ex}")

        if self.supports_chain_of_thought and use_chain_of_thought:
            parts.append("\nThink through this step by step before providing your final answer.")

        return "\n".join(parts)


# ── Available templates ──────────────────────────────────────────────────────
PROMPT_TEMPLATES: Dict[str, PromptTemplate] = {
    "standard": PromptTemplate(
        id="standard",
        name="Standard Assistant",
        system_instruction="You are a helpful AI assistant. Answer the user's question accurately and helpfully.",
    ),
    "expert": PromptTemplate(
        id="expert",
        name="Domain Expert",
        system_instruction="You are an expert in the relevant domain. Provide authoritative, well-researched answers with references where appropriate.",
    ),
    "teacher": PromptTemplate(
        id="teacher",
        name="Patient Teacher",
        system_instruction="You are a patient teacher. Explain concepts clearly, use analogies, and check for understanding.",
    ),
    "concise": PromptTemplate(
        id="concise",
        name="Concise Responder",
        system_instruction="You are a concise assistant. Provide direct, no-fluff answers.",
        supports_chain_of_thought=False,
    ),
    "creative": PromptTemplate(
        id="creative",
        name="Creative Writer",
        system_instruction="You are a creative assistant. Think outside the box and provide imaginative, engaging responses.",
    ),
    "analyst": PromptTemplate(
        id="analyst",
        name="Data Analyst",
        system_instruction="You are a data analyst. Approach problems systematically with structured, data-driven insights.",
    ),
}

print(f"✅ Defined {len(PROMPT_TEMPLATES)} prompt templates:")
for tid, t in PROMPT_TEMPLATES.items():
    print(f"   📝 {t.name} ({tid})")

## 4. LLM-as-Judge Reward Model

Instead of simulated rewards, we use a **separate LLM call** to evaluate response quality on multiple axes.
The judge LLM scores the response and we combine that with real token-cost data.

In [ ]:
class QualityScores(BaseModel):
    """Pydantic model for structured judge output."""
    relevance: float = Field(ge=0, le=1, description="How relevant the response is to the query")
    accuracy: float = Field(ge=0, le=1, description="Factual accuracy of the response")
    helpfulness: float = Field(ge=0, le=1, description="How helpful the response is")
    tone_match: float = Field(ge=0, le=1, description="How well the tone matches the requested style")
    detail_match: float = Field(ge=0, le=1, description="How well the detail level matches the request")
    reasoning: str = Field(description="Brief explanation of the scores")


JUDGE_SYSTEM_PROMPT = """You are an expert quality evaluator for AI-generated responses.
You will receive:
- The original user query
- The system prompt used to generate the response
- The AI-generated response
- The requested tone and detail level

Score the response on each dimension from 0.0 to 1.0.

Return ONLY valid JSON matching this schema:
{{
  "relevance": <float 0-1>,
  "accuracy": <float 0-1>,
  "helpfulness": <float 0-1>,
  "tone_match": <float 0-1>,
  "detail_match": <float 0-1>,
  "reasoning": "<brief explanation>"
}}

Be fair but critical. Only give 1.0 for truly excellent performance on that dimension."""


class LLMJudge:
    """
    Uses a separate LLM call to evaluate the quality of a response.
    Produces a composite reward signal combining quality scores and cost.
    """

    def __init__(self, llm_service: LLMService, judge_provider: LLMProvider = LLMProvider.OPENAI):
        self.llm_service = llm_service
        self.judge_provider = judge_provider
        self.evaluation_log: List[Dict[str, Any]] = []

    def evaluate(
        self,
        query: str,
        system_prompt_used: str,
        response: str,
        requested_tone: str,
        requested_detail: str,
    ) -> QualityScores:
        """Ask the judge LLM to score the response."""
        eval_message = (
            f"## User Query\n{query}\n\n"
            f"## System Prompt Used\n{system_prompt_used[:500]}\n\n"
            f"## AI Response\n{response[:1500]}\n\n"
            f"## Requested Style\n- Tone: {requested_tone}\n- Detail level: {requested_detail}\n\n"
            f"Evaluate the response now."
        )

        judge_resp = self.llm_service.call(
            system_prompt=JUDGE_SYSTEM_PROMPT,
            user_message=eval_message,
            provider=self.judge_provider,
        )

        try:
            # Parse JSON from judge response
            raw = judge_resp.content.strip()
            # Handle markdown code fences if present
            if raw.startswith("```"):
                raw = raw.split("```")[1]
                if raw.startswith("json"):
                    raw = raw[4:]
            scores = QualityScores(**json.loads(raw))
        except Exception:
            # Fallback: neutral scores
            scores = QualityScores(
                relevance=0.5, accuracy=0.5, helpfulness=0.5,
                tone_match=0.5, detail_match=0.5,
                reasoning="Judge parsing failed; using default scores.",
            )

        self.evaluation_log.append({
            "relevance": scores.relevance,
            "accuracy": scores.accuracy,
            "helpfulness": scores.helpfulness,
            "tone_match": scores.tone_match,
            "detail_match": scores.detail_match,
        })

        return scores

    @staticmethod
    def composite_reward(
        scores: QualityScores,
        token_cost: float,
        latency: float,
        cost_weight: float = 0.15,
        latency_weight: float = 0.05,
    ) -> float:
        """
        Compute a single scalar reward from quality scores and efficiency metrics.

        reward = quality_mean - cost_weight * normalized_cost - latency_weight * normalized_latency
        """
        quality = np.mean([
            scores.relevance,
            scores.accuracy,
            scores.helpfulness,
            scores.tone_match,
            scores.detail_match,
        ])
        # Normalize cost (assume ~$0.001 per call for gpt-4o-mini is 'normal')
        norm_cost = min(token_cost / 0.002, 1.0)
        # Normalize latency (assume 2s is 'normal')
        norm_latency = min(latency / 4.0, 1.0)

        reward = quality - cost_weight * norm_cost - latency_weight * norm_latency
        return float(np.clip(reward, 0, 1))


judge = LLMJudge(llm_service, judge_provider=LLMProvider.OPENAI)
print("✅ LLM-as-Judge reward model ready")

## 5. Query Bank and User Profiles

We define a set of real queries (sent to the LLM) and user profiles that encode preferences for the RL agents to learn.

In [ ]:
@dataclass
class Query:
    """A user query with metadata."""
    text: str
    query_type: QueryType
    complexity: float  # 0-1
    requires_examples: bool = False
    requires_reasoning: bool = False


@dataclass
class UserProfile:
    """User profile encoding preferences (used in state representation)."""
    user_id: str
    preferred_tone: ToneStyle
    preferred_detail: DetailLevel
    preferred_templates: List[str]
    expertise_level: float = 0.5  # 0-1


# ── Query bank ────────────────────────────────────────────────────────────────
QUERY_BANK: List[Query] = [
    # Factual
    Query("What is the difference between TCP and UDP?", QueryType.FACTUAL, 0.4),
    Query("Explain how transformers work in deep learning.", QueryType.FACTUAL, 0.7, requires_reasoning=True),
    Query("What causes the northern lights?", QueryType.FACTUAL, 0.3),
    # Creative
    Query("Write a short poem about the beauty of mathematics.", QueryType.CREATIVE, 0.5),
    Query("Come up with three creative names for a coffee shop run by robots.", QueryType.CREATIVE, 0.3),
    # Analytical
    Query("Compare the pros and cons of microservices vs monolithic architecture.", QueryType.ANALYTICAL, 0.6, requires_reasoning=True),
    Query("What are the trade-offs between batch and stream processing?", QueryType.ANALYTICAL, 0.7, requires_reasoning=True),
    # Coding
    Query("Write a Python function that implements binary search.", QueryType.CODING, 0.4, requires_examples=True),
    Query("Show how to implement a simple LRU cache in Python.", QueryType.CODING, 0.6, requires_examples=True),
    # Conversational
    Query("What are some good strategies for staying productive while working remotely?", QueryType.CONVERSATIONAL, 0.3),
    Query("I'm feeling overwhelmed with my workload. Any advice?", QueryType.CONVERSATIONAL, 0.2),
]

# ── User profiles ─────────────────────────────────────────────────────────────
USER_PROFILES: Dict[str, UserProfile] = {
    "tech_expert": UserProfile(
        user_id="tech_expert",
        preferred_tone=ToneStyle.TECHNICAL,
        preferred_detail=DetailLevel.DETAILED,
        preferred_templates=["expert", "analyst"],
        expertise_level=0.9,
    ),
    "casual_user": UserProfile(
        user_id="casual_user",
        preferred_tone=ToneStyle.FRIENDLY,
        preferred_detail=DetailLevel.MODERATE,
        preferred_templates=["standard", "teacher"],
        expertise_level=0.3,
    ),
    "busy_pro": UserProfile(
        user_id="busy_pro",
        preferred_tone=ToneStyle.FORMAL,
        preferred_detail=DetailLevel.BRIEF,
        preferred_templates=["concise", "expert"],
        expertise_level=0.7,
    ),
    "student": UserProfile(
        user_id="student",
        preferred_tone=ToneStyle.FRIENDLY,
        preferred_detail=DetailLevel.COMPREHENSIVE,
        preferred_templates=["teacher", "standard"],
        expertise_level=0.4,
    ),
}

print(f"✅ Query bank: {len(QUERY_BANK)} queries")
print(f"✅ User profiles: {len(USER_PROFILES)} profiles")
for uid, up in USER_PROFILES.items():
    print(f"   👤 {uid}: tone={up.preferred_tone.value}, detail={up.preferred_detail.value}")

## 6. Real LLM Environment

The environment wraps the LLM service and the judge to:
1. Build a system prompt from the RL-selected configuration
2. Call the LLM with the user query
3. Score the response with the judge
4. Return a composite reward

In [ ]:
@dataclass
class PromptAction:
    """A complete prompt configuration selected by the RL agent."""
    template_id: str
    tone: ToneStyle
    detail: DetailLevel
    use_examples: bool
    use_chain_of_thought: bool
    provider: LLMProvider = LLMProvider.OPENAI

    def to_tuple(self) -> tuple:
        return (
            self.template_id,
            self.tone.value,
            self.detail.value,
            self.use_examples,
            self.use_chain_of_thought,
            self.provider.value,
        )

    @classmethod
    def from_tuple(cls, t: tuple) -> "PromptAction":
        return cls(
            template_id=t[0],
            tone=ToneStyle(t[1]),
            detail=DetailLevel(t[2]),
            use_examples=t[3],
            use_chain_of_thought=t[4],
            provider=LLMProvider(t[5]) if len(t) > 5 else LLMProvider.OPENAI,
        )


class RealLLMEnvironment:
    """
    Environment that makes real LLM calls and evaluates responses via an LLM judge.
    """

    def __init__(
        self,
        llm_service: LLMService,
        judge: LLMJudge,
        templates: Dict[str, PromptTemplate],
        queries: List[Query],
        user_profiles: Dict[str, UserProfile],
        cost_weight: float = 0.15,
        latency_weight: float = 0.05,
    ):
        self.llm_service = llm_service
        self.judge = judge
        self.templates = templates
        self.queries = queries
        self.user_profiles = user_profiles
        self.cost_weight = cost_weight
        self.latency_weight = latency_weight
        self.interaction_history: List[Dict[str, Any]] = []

    def get_random_query(self) -> Tuple[Query, str]:
        query = random.choice(self.queries)
        user_id = random.choice(list(self.user_profiles.keys()))
        return query, user_id

    def step(self, query: Query, user_id: str, action: PromptAction) -> Tuple[float, Dict[str, Any]]:
        """
        Execute one full RL step:
          1. Build system prompt from action
          2. Call LLM
          3. Judge the response
          4. Compute composite reward
        Returns (reward, info_dict).
        """
        template = self.templates[action.template_id]

        # 1) Render system prompt
        system_prompt = template.render_system_prompt(
            tone=action.tone,
            detail=action.detail,
            use_chain_of_thought=action.use_chain_of_thought,
        )

        # 2) Call the LLM
        llm_response = self.llm_service.call(
            system_prompt=system_prompt,
            user_message=query.text,
            provider=action.provider,
        )

        # 3) Judge the response
        scores = self.judge.evaluate(
            query=query.text,
            system_prompt_used=system_prompt,
            response=llm_response.content,
            requested_tone=action.tone.value,
            requested_detail=action.detail.value,
        )

        # 4) Composite reward
        reward = LLMJudge.composite_reward(
            scores,
            token_cost=llm_response.cost_estimate,
            latency=llm_response.latency_seconds,
            cost_weight=self.cost_weight,
            latency_weight=self.latency_weight,
        )

        info = {
            "query": query.text,
            "query_type": query.query_type.value,
            "user_id": user_id,
            "template": action.template_id,
            "tone": action.tone.value,
            "detail": action.detail.value,
            "use_examples": action.use_examples,
            "use_cot": action.use_chain_of_thought,
            "provider": action.provider.value,
            "response_preview": llm_response.content[:200],
            "input_tokens": llm_response.input_tokens,
            "output_tokens": llm_response.output_tokens,
            "latency": llm_response.latency_seconds,
            "cost": llm_response.cost_estimate,
            "relevance": scores.relevance,
            "accuracy": scores.accuracy,
            "helpfulness": scores.helpfulness,
            "tone_match": scores.tone_match,
            "detail_match": scores.detail_match,
            "reward": reward,
        }
        self.interaction_history.append(info)
        return reward, info


# Create the real environment
env = RealLLMEnvironment(
    llm_service=llm_service,
    judge=judge,
    templates=PROMPT_TEMPLATES,
    queries=QUERY_BANK,
    user_profiles=USER_PROFILES,
)
print("✅ Real LLM environment created")

## 7. RL Algorithms: Multi-Armed Bandits

Identical bandit algorithms to the simulated notebook — the only change is that rewards now come from real LLM calls.

In [ ]:
class MultiArmedBandit(ABC):
    """Abstract base class for bandit algorithms."""

    def __init__(self, n_arms: int, arm_names: List[str] = None):
        self.n_arms = n_arms
        self.arm_names = arm_names or [f"arm_{i}" for i in range(n_arms)]
        self.counts = np.zeros(n_arms)
        self.values = np.zeros(n_arms)
        self.history: List[Dict] = []

    @abstractmethod
    def select_arm(self) -> int:
        pass

    def update(self, arm: int, reward: float):
        self.counts[arm] += 1
        n = self.counts[arm]
        self.values[arm] += (reward - self.values[arm]) / n
        self.history.append({"arm": arm, "reward": reward})

    def get_best_arm(self) -> int:
        return int(np.argmax(self.values))


class EpsilonGreedyBandit(MultiArmedBandit):
    def __init__(self, n_arms: int, epsilon: float = 0.1, arm_names: List[str] = None):
        super().__init__(n_arms, arm_names)
        self.epsilon = epsilon

    def select_arm(self) -> int:
        if np.random.random() < self.epsilon:
            return np.random.randint(self.n_arms)
        return self.get_best_arm()


class UCBBandit(MultiArmedBandit):
    def __init__(self, n_arms: int, c: float = 2.0, arm_names: List[str] = None):
        super().__init__(n_arms, arm_names)
        self.c = c
        self.total_counts = 0

    def select_arm(self) -> int:
        self.total_counts += 1
        for arm in range(self.n_arms):
            if self.counts[arm] == 0:
                return arm
        ucb = self.values + self.c * np.sqrt(np.log(self.total_counts) / self.counts)
        return int(np.argmax(ucb))


class ThompsonSamplingBandit(MultiArmedBandit):
    def __init__(self, n_arms: int, arm_names: List[str] = None):
        super().__init__(n_arms, arm_names)
        self.alpha = np.ones(n_arms)
        self.beta_param = np.ones(n_arms)

    def select_arm(self) -> int:
        samples = np.random.beta(self.alpha, self.beta_param)
        return int(np.argmax(samples))

    def update(self, arm: int, reward: float):
        super().update(arm, reward)
        if reward > 0.5:
            self.alpha[arm] += 1
        else:
            self.beta_param[arm] += 1


class PromptTemplateBandit:
    """Bandit-based template selector: one bandit per query type."""

    def __init__(self, templates: Dict[str, PromptTemplate], algorithm: str = "thompson"):
        self.template_ids = list(templates.keys())
        n = len(self.template_ids)
        self.bandits: Dict[QueryType, MultiArmedBandit] = {}
        for qt in QueryType:
            if algorithm == "epsilon_greedy":
                self.bandits[qt] = EpsilonGreedyBandit(n, 0.15, self.template_ids)
            elif algorithm == "ucb":
                self.bandits[qt] = UCBBandit(n, 2.0, self.template_ids)
            else:
                self.bandits[qt] = ThompsonSamplingBandit(n, self.template_ids)

    def select_template(self, query_type: QueryType) -> str:
        arm = self.bandits[query_type].select_arm()
        return self.template_ids[arm]

    def update(self, query_type: QueryType, template_id: str, reward: float):
        arm = self.template_ids.index(template_id)
        self.bandits[query_type].update(arm, reward)

    def get_best_templates(self) -> Dict[str, str]:
        return {
            qt.value: self.template_ids[self.bandits[qt].get_best_arm()]
            for qt in QueryType
        }


print("✅ Bandit algorithms defined: Epsilon-Greedy, UCB, Thompson Sampling")

## 8. RL Algorithm: Q-Learning for Full Prompt Configuration

Q-Learning optimizes the *entire* prompt configuration (template + tone + detail + chain-of-thought + provider).

In [ ]:
class PromptQLearning:
    """Q-Learning agent for full prompt configuration optimization."""

    def __init__(
        self,
        templates: Dict[str, PromptTemplate],
        learning_rate: float = 0.1,
        discount_factor: float = 0.95,
        epsilon: float = 0.3,
        epsilon_decay: float = 0.99,
        min_epsilon: float = 0.05,
        include_provider: bool = True,
    ):
        self.templates = templates
        self.template_ids = list(templates.keys())
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.min_epsilon = min_epsilon
        self.include_provider = include_provider

        self.q_table: Dict[str, Dict[tuple, float]] = defaultdict(lambda: defaultdict(float))
        self.actions = self._generate_actions()
        self.reward_history: List[float] = []
        self.epsilon_history: List[float] = []

    def _generate_actions(self) -> List[PromptAction]:
        actions = []
        providers = [LLMProvider.OPENAI, LLMProvider.ANTHROPIC] if self.include_provider else [LLMProvider.OPENAI]
        for tid in self.template_ids:
            tmpl = self.templates[tid]
            for tone in ToneStyle:
                for detail in DetailLevel:
                    for cot in [True, False]:
                        if cot and not tmpl.supports_chain_of_thought:
                            continue
                        for prov in providers:
                            actions.append(PromptAction(
                                template_id=tid,
                                tone=tone,
                                detail=detail,
                                use_examples=False,  # simplified
                                use_chain_of_thought=cot,
                                provider=prov,
                            ))
        return actions

    def get_state(self, query: Query, user_id: str = None) -> str:
        if user_id:
            return f"{query.query_type.value}_{user_id}"
        return query.query_type.value

    def select_action(self, state: str, training: bool = True) -> PromptAction:
        if training and np.random.random() < self.epsilon:
            return random.choice(self.actions)
        state_q = self.q_table[state]
        if not state_q:
            return random.choice(self.actions)
        best = max(state_q.keys(), key=lambda a: state_q[a])
        return PromptAction.from_tuple(best)

    def update(self, state: str, action: PromptAction, reward: float, next_state: str = None):
        at = action.to_tuple()
        old = self.q_table[state][at]
        max_next = max(self.q_table[next_state].values()) if (next_state and self.q_table[next_state]) else 0
        self.q_table[state][at] = old + self.learning_rate * (
            reward + self.discount_factor * max_next - old
        )
        self.epsilon = max(self.min_epsilon, self.epsilon * self.epsilon_decay)
        self.reward_history.append(reward)
        self.epsilon_history.append(self.epsilon)

    def get_best_action(self, state: str) -> Optional[PromptAction]:
        sq = self.q_table[state]
        if not sq:
            return None
        return PromptAction.from_tuple(max(sq.keys(), key=lambda a: sq[a]))

    def get_policy_summary(self) -> pd.DataFrame:
        rows = []
        for state, actions in self.q_table.items():
            if actions:
                best_t = max(actions.keys(), key=lambda a: actions[a])
                a = PromptAction.from_tuple(best_t)
                rows.append({
                    "state": state,
                    "template": a.template_id,
                    "tone": a.tone.value,
                    "detail": a.detail.value,
                    "cot": a.use_chain_of_thought,
                    "provider": a.provider.value,
                    "q_value": round(actions[best_t], 4),
                })
        return pd.DataFrame(rows)


print(f"✅ Q-Learning agent defined")
print(f"   Action space: {len(PromptQLearning(PROMPT_TEMPLATES).actions)} configurations")

## 9. LangGraph Workflow: RL Prompt Optimization Loop

We use **LangGraph** to orchestrate the RL optimization loop as a state machine:

```
START → select_action → call_llm → judge_response → compute_reward → (loop or END)
```

In [ ]:
class RLWorkflowState(TypedDict):
    """State flowing through the LangGraph optimization loop."""
    # Input
    query: str
    query_type: str
    user_id: str
    # RL agent picks
    template_id: str
    tone: str
    detail: str
    use_cot: bool
    provider: str
    # LLM response
    system_prompt: str
    response: str
    input_tokens: int
    output_tokens: int
    latency: float
    cost: float
    # Judge scores
    relevance: float
    accuracy: float
    helpfulness: float
    tone_match: float
    detail_match: float
    # Reward
    reward: float
    # Iteration tracking
    iteration: int
    max_iterations: int
    all_rewards: list


def build_rl_workflow(
    rl_agent,  # PromptQLearning or PromptTemplateBandit
    llm_service: LLMService,
    judge: LLMJudge,
    templates: Dict[str, PromptTemplate],
    queries: List[Query],
    user_profiles: Dict[str, UserProfile],
    agent_type: str = "qlearning",  # or "bandit"
):
    """
    Build a LangGraph state machine that runs the RL training loop.
    """

    def select_action_node(state: RLWorkflowState) -> dict:
        """RL agent selects a prompt configuration."""
        query = random.choice(queries)
        user_id = random.choice(list(user_profiles.keys()))

        if agent_type == "qlearning":
            q_state = rl_agent.get_state(query, user_id)
            action = rl_agent.select_action(q_state, training=True)
            return {
                "query": query.text,
                "query_type": query.query_type.value,
                "user_id": user_id,
                "template_id": action.template_id,
                "tone": action.tone.value,
                "detail": action.detail.value,
                "use_cot": action.use_chain_of_thought,
                "provider": action.provider.value,
            }
        else:  # bandit
            template_id = rl_agent.select_template(query.query_type)
            return {
                "query": query.text,
                "query_type": query.query_type.value,
                "user_id": user_id,
                "template_id": template_id,
                "tone": ToneStyle.FORMAL.value,
                "detail": DetailLevel.MODERATE.value,
                "use_cot": query.requires_reasoning,
                "provider": LLMProvider.OPENAI.value,
            }

    def call_llm_node(state: RLWorkflowState) -> dict:
        """Call the real LLM with the selected prompt configuration."""
        template = templates[state["template_id"]]
        system_prompt = template.render_system_prompt(
            tone=ToneStyle(state["tone"]),
            detail=DetailLevel(state["detail"]),
            use_chain_of_thought=state["use_cot"],
        )
        resp = llm_service.call(
            system_prompt=system_prompt,
            user_message=state["query"],
            provider=LLMProvider(state["provider"]),
        )
        return {
            "system_prompt": system_prompt,
            "response": resp.content,
            "input_tokens": resp.input_tokens,
            "output_tokens": resp.output_tokens,
            "latency": resp.latency_seconds,
            "cost": resp.cost_estimate,
        }

    def judge_response_node(state: RLWorkflowState) -> dict:
        """Use the LLM judge to score the response."""
        scores = judge.evaluate(
            query=state["query"],
            system_prompt_used=state["system_prompt"],
            response=state["response"],
            requested_tone=state["tone"],
            requested_detail=state["detail"],
        )
        return {
            "relevance": scores.relevance,
            "accuracy": scores.accuracy,
            "helpfulness": scores.helpfulness,
            "tone_match": scores.tone_match,
            "detail_match": scores.detail_match,
        }

    def compute_reward_node(state: RLWorkflowState) -> dict:
        """Compute composite reward and update RL agent."""
        scores = QualityScores(
            relevance=state["relevance"],
            accuracy=state["accuracy"],
            helpfulness=state["helpfulness"],
            tone_match=state["tone_match"],
            detail_match=state["detail_match"],
            reasoning="",
        )
        reward = LLMJudge.composite_reward(
            scores,
            token_cost=state["cost"],
            latency=state["latency"],
        )

        # Update the RL agent
        if agent_type == "qlearning":
            q_query = Query(state["query"], QueryType(state["query_type"]), 0.5)
            q_state = rl_agent.get_state(q_query, state["user_id"])
            action = PromptAction(
                template_id=state["template_id"],
                tone=ToneStyle(state["tone"]),
                detail=DetailLevel(state["detail"]),
                use_examples=False,
                use_chain_of_thought=state["use_cot"],
                provider=LLMProvider(state["provider"]),
            )
            rl_agent.update(q_state, action, reward)
        else:
            rl_agent.update(
                QueryType(state["query_type"]),
                state["template_id"],
                reward,
            )

        prev_rewards = state.get("all_rewards", []) or []

        # Print progress
        it = state["iteration"] + 1
        print(f"   [{it}/{state['max_iterations']}] reward={reward:.3f} "
              f"template={state['template_id']} tone={state['tone']} "
              f"detail={state['detail']} provider={state['provider']}")

        return {
            "reward": reward,
            "iteration": it,
            "all_rewards": prev_rewards + [reward],
        }

    def should_continue(state: RLWorkflowState) -> Literal["select_action", "__end__"]:
        if state["iteration"] >= state["max_iterations"]:
            return "__end__"
        return "select_action"

    # ── Build the graph ────────────────────────────────────────────────────
    workflow = StateGraph(RLWorkflowState)
    workflow.add_node("select_action", select_action_node)
    workflow.add_node("call_llm", call_llm_node)
    workflow.add_node("judge_response", judge_response_node)
    workflow.add_node("compute_reward", compute_reward_node)

    workflow.add_edge(START, "select_action")
    workflow.add_edge("select_action", "call_llm")
    workflow.add_edge("call_llm", "judge_response")
    workflow.add_edge("judge_response", "compute_reward")
    workflow.add_conditional_edges("compute_reward", should_continue)

    return workflow.compile()


print("✅ LangGraph RL workflow builder ready")

## 10. Training with Real LLM Calls

We train both **Thompson Sampling bandit** and **Q-Learning** agents using the LangGraph workflow.

> ⚠️ **Cost note:** Each iteration makes **2 LLM calls** (one generation + one judge). We keep iterations modest to control costs. With `gpt-4o-mini` this typically costs < \$0.10 for 30 iterations.

In [ ]:
# ── Configuration ─────────────────────────────────────────────────────────────
N_BANDIT_ITERS = 30   # Thompson Sampling training iterations
N_QLEARN_ITERS = 30   # Q-Learning training iterations

print(f"📋 Training plan:")
print(f"   Bandit iterations:     {N_BANDIT_ITERS}")
print(f"   Q-Learning iterations: {N_QLEARN_ITERS}")
print(f"   Total LLM calls:       ~{(N_BANDIT_ITERS + N_QLEARN_ITERS) * 2} (generation + judge)")
print(f"   Estimated cost:        ~${(N_BANDIT_ITERS + N_QLEARN_ITERS) * 2 * 0.0005:.2f} (gpt-4o-mini)")

In [ ]:
# ── Train Thompson Sampling Bandit ────────────────────────────────────────────
print("\n🎰 Training Thompson Sampling Bandit...")
print("=" * 60)

bandit_agent = PromptTemplateBandit(PROMPT_TEMPLATES, algorithm="thompson")

bandit_workflow = build_rl_workflow(
    rl_agent=bandit_agent,
    llm_service=llm_service,
    judge=judge,
    templates=PROMPT_TEMPLATES,
    queries=QUERY_BANK,
    user_profiles=USER_PROFILES,
    agent_type="bandit",
)

initial_state = {
    "iteration": 0,
    "max_iterations": N_BANDIT_ITERS,
    "all_rewards": [],
    "query": "", "query_type": "", "user_id": "",
    "template_id": "", "tone": "", "detail": "",
    "use_cot": False, "provider": "",
    "system_prompt": "", "response": "",
    "input_tokens": 0, "output_tokens": 0,
    "latency": 0.0, "cost": 0.0,
    "relevance": 0.0, "accuracy": 0.0,
    "helpfulness": 0.0, "tone_match": 0.0,
    "detail_match": 0.0, "reward": 0.0,
}

bandit_result = bandit_workflow.invoke(initial_state)

bandit_rewards = bandit_result["all_rewards"]
print(f"\n✅ Bandit training complete: {len(bandit_rewards)} iterations")
print(f"   Mean reward: {np.mean(bandit_rewards):.4f}")
print(f"   Best templates learned: {bandit_agent.get_best_templates()}")

In [ ]:
# ── Train Q-Learning Agent ────────────────────────────────────────────────────
print("\n🧠 Training Q-Learning Agent...")
print("=" * 60)

qlearn_agent = PromptQLearning(
    PROMPT_TEMPLATES,
    learning_rate=0.15,
    epsilon=0.4,
    epsilon_decay=0.97,
    min_epsilon=0.1,
    include_provider=True,
)

qlearn_workflow = build_rl_workflow(
    rl_agent=qlearn_agent,
    llm_service=llm_service,
    judge=judge,
    templates=PROMPT_TEMPLATES,
    queries=QUERY_BANK,
    user_profiles=USER_PROFILES,
    agent_type="qlearning",
)

qlearn_result = qlearn_workflow.invoke({
    "iteration": 0,
    "max_iterations": N_QLEARN_ITERS,
    "all_rewards": [],
    "query": "", "query_type": "", "user_id": "",
    "template_id": "", "tone": "", "detail": "",
    "use_cot": False, "provider": "",
    "system_prompt": "", "response": "",
    "input_tokens": 0, "output_tokens": 0,
    "latency": 0.0, "cost": 0.0,
    "relevance": 0.0, "accuracy": 0.0,
    "helpfulness": 0.0, "tone_match": 0.0,
    "detail_match": 0.0, "reward": 0.0,
})

qlearn_rewards = qlearn_result["all_rewards"]
print(f"\n✅ Q-Learning training complete: {len(qlearn_rewards)} iterations")
print(f"   Mean reward: {np.mean(qlearn_rewards):.4f}")
print(f"   Final epsilon: {qlearn_agent.epsilon:.4f}")

In [ ]:
# ── Cost summary so far ───────────────────────────────────────────────────────
cost_summary = llm_service.get_cost_summary()
print("\n💰 Cost Summary After Training")
print("=" * 60)
for k, v in cost_summary.items():
    print(f"   {k}: {v}")

## 11. Results Visualization

In [ ]:
def moving_average(data, window=5):
    return pd.Series(data).rolling(window=window, min_periods=1).mean()


fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle("RL Prompt Optimization — Real LLM Calls", fontsize=16, fontweight="bold")

# ── Plot 1: Reward curves ─────────────────────────────────────────────────────
ax = axes[0, 0]
if bandit_rewards:
    ax.plot(moving_average(bandit_rewards), label="Thompson Sampling", marker=".", markersize=3)
if qlearn_rewards:
    ax.plot(moving_average(qlearn_rewards), label="Q-Learning", marker=".", markersize=3)
ax.set_xlabel("Iteration")
ax.set_ylabel("Reward (Moving Avg)")
ax.set_title("Training Reward Curves")
ax.legend()
ax.grid(True, alpha=0.3)

# ── Plot 2: Per-iteration reward comparison ───────────────────────────────────
ax = axes[0, 1]
methods = ["Thompson\nSampling", "Q-Learning"]
means = [np.mean(bandit_rewards) if bandit_rewards else 0,
         np.mean(qlearn_rewards) if qlearn_rewards else 0]
stds = [np.std(bandit_rewards) if bandit_rewards else 0,
        np.std(qlearn_rewards) if qlearn_rewards else 0]
colors = ["#2ecc71", "#9b59b6"]
bars = ax.bar(methods, means, yerr=stds, color=colors, edgecolor="black", capsize=5)
for bar, val in zip(bars, means):
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.02,
            f"{val:.3f}", ha="center", fontsize=11)
ax.set_ylabel("Mean Reward")
ax.set_title("Average Reward Comparison")
ax.grid(True, alpha=0.3, axis="y")

# ── Plot 3: Judge quality breakdown (recent interactions) ─────────────────────
ax = axes[1, 0]
if judge.evaluation_log:
    last_n = min(20, len(judge.evaluation_log))
    eval_df = pd.DataFrame(judge.evaluation_log[-last_n:])
    dims = ["relevance", "accuracy", "helpfulness", "tone_match", "detail_match"]
    means_q = [eval_df[d].mean() for d in dims]
    ax.barh(dims, means_q, color=sns.color_palette("viridis", len(dims)), edgecolor="black")
    for i, v in enumerate(means_q):
        ax.text(v + 0.01, i, f"{v:.2f}", va="center", fontsize=10)
    ax.set_xlim(0, 1.1)
ax.set_title("Judge Quality Scores (recent)")
ax.set_xlabel("Score (0-1)")

# ── Plot 4: Cumulative cost ───────────────────────────────────────────────────
ax = axes[1, 1]
if llm_service.call_log:
    cum_cost = np.cumsum([c["cost"] for c in llm_service.call_log])
    ax.plot(cum_cost, color="#e74c3c", linewidth=2)
    ax.fill_between(range(len(cum_cost)), cum_cost, alpha=0.15, color="#e74c3c")
    ax.set_xlabel("LLM Call #")
    ax.set_ylabel("Cumulative Cost (USD)")
    ax.set_title(f"Total Cost: ${cum_cost[-1]:.4f}")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 12. Analyzing Learned Policies

In [ ]:
# ── Bandit learned preferences ────────────────────────────────────────────────
print("🎰 Thompson Sampling Bandit — Learned Template Preferences")
print("=" * 60)
best_map = bandit_agent.get_best_templates()
for qt, tid in best_map.items():
    print(f"   {qt:<18} → {PROMPT_TEMPLATES[tid].name}")

# ── Q-Learning learned policy ─────────────────────────────────────────────────
print("\n🧠 Q-Learning — Learned Prompt Configurations")
print("=" * 60)
policy_df = qlearn_agent.get_policy_summary()
if not policy_df.empty:
    print(policy_df.to_string(index=False))
else:
    print("   (No policy entries — increase training iterations)")

In [ ]:
# ── Policy heatmap ────────────────────────────────────────────────────────────
if not policy_df.empty:
    fig, ax = plt.subplots(figsize=(10, 4))
    query_types = [qt.value for qt in QueryType]
    template_ids = list(PROMPT_TEMPLATES.keys())

    pref_matrix = np.zeros((len(query_types), len(template_ids)))
    for i, qt in enumerate(QueryType):
        state = qt.value
        if state in qlearn_agent.q_table:
            for at, qv in qlearn_agent.q_table[state].items():
                j = template_ids.index(at[0])
                pref_matrix[i, j] = max(pref_matrix[i, j], qv)

    sns.heatmap(
        pref_matrix, xticklabels=template_ids, yticklabels=query_types,
        annot=True, fmt=".3f", cmap="YlGnBu", ax=ax,
    )
    ax.set_title("Q-Learning: Template Preference by Query Type")
    ax.set_xlabel("Template")
    ax.set_ylabel("Query Type")
    plt.tight_layout()
    plt.show()
else:
    print("Skipping heatmap — no Q-table entries.")

## 13. Live Demo: Optimized Prompt Generation

Use the trained agents to generate optimized prompts and get real LLM responses.

In [ ]:
def demo_optimized_prompt(
    query_text: str,
    query_type: QueryType,
    user_id: str,
    agent: PromptQLearning,
    llm_svc: LLMService,
    templates: Dict[str, PromptTemplate],
) -> None:
    """Run a single optimized query and display results."""
    q = Query(query_text, query_type, 0.5)
    state = agent.get_state(q, user_id)
    action = agent.select_action(state, training=False)

    tmpl = templates[action.template_id]
    sys_prompt = tmpl.render_system_prompt(
        tone=action.tone,
        detail=action.detail,
        use_chain_of_thought=action.use_chain_of_thought,
    )

    resp = llm_svc.call(sys_prompt, query_text, provider=action.provider)

    print(f"📝 Query: {query_text}")
    print(f"   👤 User: {user_id}")
    print(f"   🎯 Config: template={action.template_id} | tone={action.tone.value} "
          f"| detail={action.detail.value} | cot={action.use_chain_of_thought} "
          f"| provider={action.provider.value}")
    print(f"   📊 Tokens: {resp.total_tokens} | Latency: {resp.latency_seconds}s "
          f"| Cost: ${resp.cost_estimate:.6f}")
    print(f"   💬 Response:\n{resp.content[:500]}")
    print("-" * 70)


print("🚀 Live Demo — Optimized Prompts via Trained Q-Learning Agent")
print("=" * 70)

demo_queries = [
    ("What is the CAP theorem?", QueryType.FACTUAL, "tech_expert"),
    ("Write a creative tagline for a green-energy startup.", QueryType.CREATIVE, "casual_user"),
    ("What are the key differences between REST and GraphQL?", QueryType.ANALYTICAL, "busy_pro"),
    ("Write a Python decorator that retries a function 3 times.", QueryType.CODING, "student"),
]

for qt, qtype, uid in demo_queries:
    demo_optimized_prompt(qt, qtype, uid, qlearn_agent, llm_service, PROMPT_TEMPLATES)
    print()

## 14. Production Integration Class

A drop-in class you can use in a real application to serve RL-optimized prompts with online learning.

In [ ]:
class ProductionPromptOptimizer:
    """
    Production-ready prompt optimizer.

    - Wraps a trained Q-Learning agent
    - Calls real LLMs via LangChain
    - Evaluates via LLM-as-judge for online learning
    - Tracks costs and quality over time
    """

    def __init__(
        self,
        templates: Dict[str, PromptTemplate],
        llm_service: LLMService,
        judge: LLMJudge,
        pretrained_agent: PromptQLearning = None,
        online_learning: bool = True,
    ):
        self.templates = templates
        self.llm_service = llm_service
        self.judge = judge
        self.agent = pretrained_agent or PromptQLearning(templates)
        self.online_learning = online_learning
        self.serving_log: List[Dict[str, Any]] = []

    def serve(
        self,
        query_text: str,
        query_type: QueryType,
        user_id: str = None,
        evaluate: bool = True,
    ) -> Dict[str, Any]:
        """
        Serve an optimized response for a user query.

        1. Agent selects optimal prompt config
        2. LLM generates response
        3. (Optional) Judge evaluates and agent updates online
        """
        q = Query(query_text, query_type, 0.5)
        state = self.agent.get_state(q, user_id)
        action = self.agent.select_action(state, training=False)

        tmpl = self.templates[action.template_id]
        sys_prompt = tmpl.render_system_prompt(
            tone=action.tone,
            detail=action.detail,
            use_chain_of_thought=action.use_chain_of_thought,
        )

        resp = self.llm_service.call(sys_prompt, query_text, provider=action.provider)

        result = {
            "response": resp.content,
            "config": {
                "template": action.template_id,
                "tone": action.tone.value,
                "detail": action.detail.value,
                "cot": action.use_chain_of_thought,
                "provider": action.provider.value,
            },
            "tokens": resp.total_tokens,
            "latency": resp.latency_seconds,
            "cost": resp.cost_estimate,
        }

        # Optional: evaluate and update online
        if evaluate and self.online_learning:
            scores = self.judge.evaluate(
                query=query_text,
                system_prompt_used=sys_prompt,
                response=resp.content,
                requested_tone=action.tone.value,
                requested_detail=action.detail.value,
            )
            reward = LLMJudge.composite_reward(
                scores, token_cost=resp.cost_estimate, latency=resp.latency_seconds
            )
            self.agent.update(state, action, reward)
            result["quality_scores"] = {
                "relevance": scores.relevance,
                "accuracy": scores.accuracy,
                "helpfulness": scores.helpfulness,
                "tone_match": scores.tone_match,
                "detail_match": scores.detail_match,
            }
            result["reward"] = reward

        self.serving_log.append(result)
        return result

    def record_human_feedback(self, query_text: str, query_type: QueryType,
                              user_id: str, action: PromptAction, score: float):
        """Record explicit human feedback for RLHF-style updates."""
        state = self.agent.get_state(Query(query_text, query_type, 0.5), user_id)
        self.agent.update(state, action, score)

    def get_stats(self) -> Dict[str, Any]:
        if not self.serving_log:
            return {"message": "No interactions yet"}
        df = pd.DataFrame(self.serving_log)
        stats = {
            "total_served": len(df),
            "total_cost_usd": round(df["cost"].sum(), 6),
            "avg_latency_s": round(df["latency"].mean(), 3),
            "avg_tokens": int(df["tokens"].mean()),
        }
        if "reward" in df.columns:
            stats["avg_reward"] = round(df["reward"].mean(), 4)
        return stats


# ── Quick demo ─────────────────────────────────────────────────────────────────
optimizer = ProductionPromptOptimizer(
    templates=PROMPT_TEMPLATES,
    llm_service=llm_service,
    judge=judge,
    pretrained_agent=qlearn_agent,
    online_learning=True,
)

result = optimizer.serve(
    query_text="Explain the concept of eventual consistency in distributed systems.",
    query_type=QueryType.FACTUAL,
    user_id="tech_expert",
)

print("🚀 Production Optimizer — Single Serve")
print("=" * 60)
print(f"Config:  {result['config']}")
print(f"Tokens:  {result['tokens']}  |  Latency: {result['latency']}s  |  Cost: ${result['cost']:.6f}")
if "reward" in result:
    print(f"Reward:  {result['reward']:.4f}")
    print(f"Quality: {result['quality_scores']}")
print(f"\nResponse:\n{result['response'][:600]}")

## 15. Final Cost Report

In [ ]:
print("💰 Final Cost Report")
print("=" * 60)
final_cost = llm_service.get_cost_summary()
for k, v in final_cost.items():
    print(f"   {k}: {v}")

print(f"\n   Optimizer serving stats: {optimizer.get_stats()}")

## 16. Summary

This notebook demonstrated **RL-based prompt optimization with real LLM calls**:

| Component | Implementation |
|-----------|---------------|
| **LLM Providers** | OpenAI (`gpt-4o-mini`) + Anthropic (`claude-3.5-haiku`) via LangChain |
| **Reward Signal** | LLM-as-Judge (5 quality dimensions) + real token cost + latency |
| **RL Algorithms** | Thompson Sampling Bandit, Q-Learning |
| **Workflow Engine** | LangGraph state machine (select → call → judge → reward → loop) |
| **Action Space** | Template × Tone × Detail × CoT × Provider |
| **Online Learning** | Continuous improvement from production feedback |

### Key Takeaways

1. **LLM-as-Judge** provides reliable automated quality signals without human labeling
2. **Thompson Sampling** is excellent for quickly learning which template works best per query type
3. **Q-Learning** discovers richer configurations (template + tone + detail + provider combos)
4. **Cost-aware rewards** naturally push the agent toward efficient configurations
5. **Online learning** means the system keeps improving from every production interaction

### Production Recommendations

- Start with **Thompson Sampling** (simpler, faster convergence)
- Graduate to **Q-Learning** when you need multi-dimensional optimization
- Add **explicit human feedback** alongside LLM-judge for RLHF-style training
- Use **A/B testing** to validate RL improvements against static baselines
- Monitor **cost per quality unit** to ensure efficiency

In [ ]:
print("\n🎉 RL Prompt Optimization with Real LLM Calls — Complete!")
print("\nThis notebook demonstrated:")
print("  • Real LLM calls via LangChain (OpenAI + Anthropic)")
print("  • LLM-as-Judge reward model with 5 quality dimensions")
print("  • Thompson Sampling bandit for template selection")
print("  • Q-Learning for full prompt configuration optimization")
print("  • LangGraph state machine for the RL training loop")
print("  • Real token cost tracking and cost-aware rewards")
print("  • Production-ready optimizer with online learning")
print(f"\n💰 Total session cost: ${llm_service.total_cost:.4f}")